In [174]:
import pandas as pd
import numpy as np
import scipy.stats as st
import seaborn as sns
import sys
import os
import matplotlib.pyplot as plt
import swan_vis as swan
import yaml
from snakemake.io import expand
import cerberus
import pyranges as pr
import upsetplot
import re

p = os.path.dirname(os.path.dirname(os.getcwd()))
sys.path.append(p)

from scripts.utils import *
from scripts.vcf_utils import *
from scripts.sm_utils import *
from scripts.plotting import *

In [169]:
config = load_config()
od = ''

def proc_cfg(entry, od):
    entry = entry.replace('../../', '')
    entry = od+entry
    return entry

## 25/01/21 making summary files

In [170]:
df = load_meta()
df = df.loc[~df['sample'].str.contains('_')]
df['lab_sample'] = df['lab_number_sample'].astype(str)+'_'+\
                      df['lab_sampleid'].astype(str)+'_'+\
                      df['cell_line_id'].astype(str)
df = df[['cell_line_id', 'sample', 'hapmap_DNA_ID', 'lab_sample']].drop_duplicates()

temp_df = pd.read_csv('cell_line_ids.txt', header=None, names=['cell_line_id'])

# make a 1000g cell line id col
df['cell_line_id_1000g'] = df.cell_line_id

inds = df.loc[~df.cell_line_id_1000g.isin(temp_df.cell_line_id.tolist())].index
df.loc[inds, 'cell_line_id_1000g'] = df.loc[inds, 'hapmap_DNA_ID']
len(df.index)

# limit to just those in 1000g
df = df.loc[df.cell_line_id_1000g.isin(temp_df.cell_line_id.tolist())]
assert len(df.index) == 30

# # TODO removing one problematic sample
# bad_samples = ['NA19328']
# df = df.loc[df.cell_line_id_1000g.isin(bad_samples)]

# # TODO try on just one cell line
# test_samples = ['NA12273']
# df = df.loc[df.cell_line_id_1000g.isin(test_samples)]

hap = ['hap1', 'hap2']

# genome information table
sqanti_genomes = ['hap1', 'hap2', 'hg38']
sqanti_df = df.copy(deep=True)
sqanti_df['sqanti_genome'] = [sqanti_genomes] * len(df)
sqanti_df = sqanti_df.explode('sqanti_genome').reset_index(drop=True)

In [171]:
len(sqanti_df)

90

In [180]:
# summary df table
map_genomes = ['hap1', 'hap2', 'hg38']
summ_df = sqanti_df.copy(deep=True)
summ_df['map_genome'] = [map_genomes] * len(summ_df)
summ_df = summ_df.explode('map_genome').reset_index(drop=True)
summ_df.head()

,cell_line_id,sample,hapmap_DNA_ID,lab_sample,cell_line_id_1000g,sqanti_genome,map_genome
0,HG04217,ITU5,no_hapmap,39_IN5_HG04217,HG04217,hap1,hap1
1,HG04217,ITU5,no_hapmap,39_IN5_HG04217,HG04217,hap1,hap2
2,HG04217,ITU5,no_hapmap,39_IN5_HG04217,HG04217,hap1,hg38
3,HG04217,ITU5,no_hapmap,39_IN5_HG04217,HG04217,hap2,hap1
4,HG04217,ITU5,no_hapmap,39_IN5_HG04217,HG04217,hap2,hap2


In [181]:
# this is code to get the file names for each input file that we need to summarize 

# # sqanti classification files
# td_sq_cl_files = expand(config['lr']['td_personal']['sqanti']['cl'],
#                    cell_line_id=df['cell_line_id_1000g'].tolist(),
#                    hap=hap,
#                    sqanti_genome=sqanti_genomes)
# hg38_sq_cl_files = expand(config['lr']['espresso']['sqanti']['cl'],
#                           cell_line_id=df['cell_line_id_1000g'].tolist(),
#                           sqanti_genome=sqanti_genomes)

# # espresso gtfs
# td_sq_cl_files = expand(config['lr']['td_personal']['sqanti']['cl'],
#                    cell_line_id=df['cell_line_id_1000g'].tolist(),
#                    hap=hap,
#                    sqanti_genome=sqanti_genomes)
# hg38_sq_cl_files = expand(config['lr']['espresso']['sqanti']['cl'],
#                           cell_line_id=df['cell_line_id_1000g'].tolist(),
#                           sqanti_genome=sqanti_genomes)

# input
# sqanti classification files
hg38_cl_files = expand(config['lr']['espresso']['sqanti']['cl'],
                          cell_line_id=df['cell_line_id_1000g'].tolist(),
                          sqanti_genome=sqanti_genomes)
td_cl_files = expand(config['lr']['td_personal']['sqanti']['cl'],
                     cell_line_id=df['cell_line_id_1000g'].tolist(),
                     hap=hap,
                     sqanti_genome=sqanti_genomes)

# sj files
hg38_sj_files = expand(config['lr']['espresso']['sqanti']['sjs'],
                          cell_line_id=df['cell_line_id_1000g'].tolist(),
                          sqanti_genome=sqanti_genomes)
td_sj_files = expand(config['lr']['td_personal']['sqanti']['sjs'],
                     cell_line_id=df['cell_line_id_1000g'].tolist(),
                     hap=hap,
                     sqanti_genome=sqanti_genomes)


# espresso gtf files
hg38_gtf_files =  expand(config['lr']['espresso']['gtf'],
                          lab_sample=df['lab_sample'].tolist())
td_gtf_files =  expand(config['lr']['td_personal']['espresso']['gtf'],
                       cell_line_id=df['cell_line_id_1000g'].tolist(),
                       hap=hap)

# actual code
ic_df = pd.DataFrame()
ss_df = pd.DataFrame()
for ind, entry in summ_df.iterrows():
    cell_line_id = entry.cell_line_id_1000g
    lab_sample = entry.lab_sample
    map_genome = entry.map_genome
    sqanti_genome = entry.sqanti_genome
    
    if entry.map_genome=='hg38':
        cl_file = expand(config['lr']['espresso']['sqanti']['cl'],
                          cell_line_id=cell_line_id,
                          sqanti_genome=sqanti_genome)[0]
        sj_file = expand(config['lr']['espresso']['sqanti']['sjs'],
                          cell_line_id=cell_line_id,
                          sqanti_genome=sqanti_genome)[0]
        gtf_file = expand(config['lr']['espresso']['gtf'],
                          lab_sample=lab_sample)[0]
                          
        
    else:
        cl_file = expand(config['lr']['td_personal']['sqanti']['cl'],
                         cell_line_id=cell_line_id,
                         sqanti_genome=sqanti_genome,
                         hap=map_genome)[0]
        sj_file = expand(config['lr']['td_personal']['sqanti']['sjs'],
                         cell_line_id=cell_line_id,
                         sqanti_genome=sqanti_genome,
                         hap=map_genome)[0]
        gtf_file = expand(config['lr']['td_personal']['espresso']['gtf'],
                          cell_line_id=cell_line_id,
                          hap=map_genome)[0]
    
    ######################
    ## IC DF
    #########################
    # get ics + tid from cerberus
    df = pr.read_gtf(gtf_file, duplicate_attr=True)
    df = df.df
    df = cerberus.add_stable_gid(df)
    df = pr.PyRanges(df)
    df = cerberus.get_ic(df)

    # remove monoexonic
    df = df.loc[df.ic!='-']

    # make sure no monoexonic
    assert len(df.loc[df.ic=='-']) == 0

    # remove gene id; we're not considering this here
    df.drop('gene_id', axis=1, inplace=True)

    # get ic_id from chr strand and ic; then drop constituent cols
    df['ic_id'] = df.Chromosome.astype(str)+'_'+\
                  df.Strand.astype(str)+'_'+\
                  df.ic.astype(str)
    df.drop(['Chromosome', 'Strand', 'ic'], axis=1, inplace=True)

    # add the classification from the sqanti file
    sq_df = pd.read_csv(cl_file, sep='\t')

    # can modify here any other details we may want from class. tables
    sq_df = sq_df[['isoform', 'structural_category']].rename({'isoform':'transcript_id'}, axis=1)
    df = df.merge(sq_df, 
                  how='left', 
                  on='transcript_id')

    # gb ic and structural cat; record tids from each 
    # make sure we're only annotating 1 structural category per ic
    df = df.groupby(['ic_id', 'structural_category']).agg(','.join).reset_index()
    assert len(df.loc[df.ic_id.duplicated(keep=False)].index) == 0

    # add sample, map/td genome, sqanti genome metadata and concat
    df['cell_line_id'] = cell_line_id
    df['map_genome'] = map_genome
    df['sqanti_genome'] = sqanti_genome
    
    df.to_csv(output.ic_summary, sep='\t', index=False)
    
    ######################
    ## SJ DF
    #########################    
    df = pd.read_csv(sj_file, sep='\t')
    df = df[['chrom', 'strand', 'genomic_start_coord', 'genomic_end_coord', 'splice_site', 'canonical']]
    df['sj_id'] = df['chrom'].astype(str)+'_'+\
                  df['strand'].astype(str)+'_'+\
                  df['genomic_start_coord'].astype(str)+'_'+\
                  df['genomic_end_coord'].astype(str)
    df = df[['sj_id', 'splice_site', 'canonical']].drop_duplicates()
    df = df.rename({'splice_site':'splice_motif'}, axis=1)
    
    # add sample, map/td genome, sqanti genome metadata and concat
    df['cell_line_id'] = cell_line_id
    df['map_genome'] = map_genome
    df['sqanti_genome'] = sqanti_genome
    
    df.to_csv(output.sj_summary, sep='\t', index=False)
    




In [191]:
# this is code to parse sss out form the sqanti sj file
sj_file = '/Users/fairliereese/Documents/programming/mele_lab/projects/240903_pt/data/td_personal/sqanti/HG04217_hap1_hap2_sqanti_junctions.txt'
df = pd.read_csv(sj_file, sep='\t')
df = df[['chrom', 'strand', 'genomic_start_coord', 'genomic_end_coord', 'splice_site', 'canonical']]
df['sj_id'] = df['chrom'].astype(str)+'_'+\
              df['strand'].astype(str)+'_'+\
              df['genomic_start_coord'].astype(str)+'_'+\
              df['genomic_end_coord'].astype(str)
df = df[['sj_id', 'splice_site', 'canonical']].drop_duplicates()
df = df.rename({'splice_site':'splice_motif'}, axis=1)

In [192]:
df.head()

,sj_id,splice_motif,canonical
0,chr3_-_195915_195995,GTAG,canonical
1,chr3_-_196041_196455,GTAG,canonical
2,chr3_-_196509_196606,GTAG,canonical
3,chr3_-_196703_196966,GTAG,canonical
4,chr3_+_197369_244612,GTAG,canonical


In [189]:
# df = df[['splice_site', 'canonical']]
# df.loc[df.canonical=='canonical'].splice_site.unique()
# df['

array(['GTAG', 'GCAG', 'ATAC'], dtype=object)

In [158]:
# this is code to parse each set of files (espresso gtf + sqanti classification)


# get ics + tid from cerberus
gtf_file = '/Users/fairliereese/Documents/programming/mele_lab/projects/240903_pt/data/espresso/samples_N2_R0_updated.gtf'
df = pr.read_gtf(gtf_file, duplicate_attr=True)
df = df.df
df = cerberus.add_stable_gid(df)
df = pr.PyRanges(df)
df = cerberus.get_ic(df)

# remove monoexonic
df = df.loc[df.ic!='-']

# make sure no monoexonic
assert len(df.loc[df.ic=='-']) == 0

# remove gene id; we're not considering this here
df.drop('gene_id', axis=1, inplace=True)

# get ic_id from chr strand and ic; then drop constituent cols
df['ic_id'] = df.Chromosome.astype(str)+'_'+\
              df.Strand.astype(str)+'_'+\
              df.ic.astype(str)
df.drop(['Chromosome', 'Strand', 'ic'], axis=1, inplace=True)

# add the classification from the sqanti file
class_file = '/Users/fairliereese/Documents/programming/mele_lab/projects/240903_pt/data/espresso/sqanti/HG02293_hap1_sqanti_classification.txt'
sq_df = pd.read_csv(class_file, sep='\t')

# can modify here any other details we may want from class. tables
sq_df = sq_df[['isoform', 'structural_category']].rename({'isoform':'transcript_id'}, axis=1)
df = df.merge(sq_df, 
              how='left', 
              on='transcript_id')

# gb ic and structural cat; record tids from each 
# make sure we're only annotating 1 structural category per ic
df = df.groupby(['ic_id', 'structural_category']).agg(','.join).reset_index()
assert len(df.loc[df.ic_id.duplicated(keep=False)].index) == 0

# TODO - add sample, map/td genome, sqanti genome metadata

In [159]:
df.head()

,transcript_id,ic_id
0,ENST00000797506.1,GL000195.1_-_86517-74180-74120-68450
1,ENST00000797519.1,GL000195.1_-_86517-84116
2,ENST00000797520.1,GL000195.1_-_86517-84277
3,ENST00000797527.1,GL000195.1_-_86517-84679
4,ESPRESSO:GL000195.1:2:0,GL000195.1_-_86517-49119-48954-43200


## older

In [126]:
# sample information
df = load_meta()
df = df.loc[~df['sample'].str.contains('_')]
df['lab_sample'] = df['lab_number_sample'].astype(str)+'_'+\
                      df['lab_sampleid'].astype(str)+'_'+\
                      df['cell_line_id'].astype(str)
df = df[['cell_line_id', 'sample', 'hapmap_DNA_ID', 'lab_sample']].drop_duplicates()

temp_df = pd.read_csv('cell_line_ids.txt', header=None, names=['cell_line_id'])

# make a 1000g cell line id col
df['cell_line_id_1000g'] = df.cell_line_id

inds = df.loc[~df.cell_line_id_1000g.isin(temp_df.cell_line_id.tolist())].index
df.loc[inds, 'cell_line_id_1000g'] = df.loc[inds, 'hapmap_DNA_ID']
len(df.index)

# limit to just those in 1000g
df = df.loc[df.cell_line_id_1000g.isin(temp_df.cell_line_id.tolist())]
assert len(df.index) == 30

hap = ['hap1', 'hap2']

# genome information table
sqanti_genomes = ['hap1', 'hap2', 'hg38']
sqanti_df = df.copy(deep=True)
sqanti_df['sqanti_genome'] = [sqanti_genomes] * len(df)
sqanti_df = sqanti_df.explode('sqanti_genome').reset_index(drop=True)


In [138]:
def get_sqanti_ref_fa(wc):
    if wc['sqanti_genome'] == 'hg38':
        return config['ref']['fa']
    else:
        return expand(config['lr']['td_personal']['ref_fa'],
                        cell_line_id=wc['cell_line_id'],
                        hap=wc['sqanti_genome'])[0]
    

In [142]:
wc = {'cell_line_id': 'HG04217', 'sqanti_genome': 'hap1'}
print(get_sqanti_ref_fa(wc))
print()

wc = {'cell_line_id': 'HG04217', 'sqanti_genome': 'hap2'}
print(get_sqanti_ref_fa(wc))
print()

wc = {'cell_line_id': 'HG04217', 'sqanti_genome': 'hg38'}
print(get_sqanti_ref_fa(wc))
print()

/gpfs/projects/bsc83/Projects/pantranscriptome/pclavell/08_allele_specifics/data/01_samples_vcf/HG04217_hap1.fa

/gpfs/projects/bsc83/Projects/pantranscriptome/pclavell/08_allele_specifics/data/01_samples_vcf/HG04217_hap2.fa

../../ref/hg38.fa



In [139]:
sqanti_df.head()

,cell_line_id,sample,hapmap_DNA_ID,lab_sample,cell_line_id_1000g,sqanti_genome
0,HG04217,ITU5,no_hapmap,39_IN5_HG04217,HG04217,hap1
1,HG04217,ITU5,no_hapmap,39_IN5_HG04217,HG04217,hap2
2,HG04217,ITU5,no_hapmap,39_IN5_HG04217,HG04217,hg38
3,HG04216,ITU4,no_hapmap,38_IN4_HG04216,HG04216,hap1
4,HG04216,ITU4,no_hapmap,38_IN4_HG04216,HG04216,hap2


In [123]:
# df

In [124]:
df = load_meta()
df = df.loc[~df['sample'].str.contains('_')]
df['lab_sample'] = df['lab_number_sample'].astype(str)+'_'+\
                      df['lab_sampleid'].astype(str)+'_'+\
                      df['cell_line_id'].astype(str)
df = df[['cell_line_id', 'sample', 'hapmap_DNA_ID', 'lab_sample']].drop_duplicates()

temp_df = pd.read_csv('cell_line_ids.txt', header=None, names=['cell_line_id'])

# make a 1000g cell line id col
df['cell_line_id_1000g'] = df.cell_line_id

inds = df.loc[~df.cell_line_id_1000g.isin(temp_df.cell_line_id.tolist())].index
df.loc[inds, 'cell_line_id_1000g'] = df.loc[inds, 'hapmap_DNA_ID']
len(df.index)

# limit to just those in 1000g
df = df.loc[df.cell_line_id_1000g.isin(temp_df.cell_line_id.tolist())]
assert len(df.index) == 30

hap = ['hap1', 'hap2']

In [125]:
bad_samples = ['NA19328']

df.loc[df.cell_line_id_1000g.isin(bad_samples)]

,cell_line_id,sample,hapmap_DNA_ID,lab_sample,cell_line_id_1000g
27,GM19328,LWK2,NA19328,20_KE2_GM19328,NA19328


In [118]:
def get_df_val(df, col1, col_dict):
    temp = df.copy(deep=True)

    for key, item in col_dict.items():
        temp = temp.loc[temp[key] == item]

    val = temp[col1].unique()
    assert len(val) == 1
    return val[0]
get_df_val(df,
                            'lab_sample',
                            {'cell_line_id_1000g': 'HG04217'})

'39_IN5_HG04217'

In [119]:
expand(config['lr']['espresso']['gtf'],
       lab_rep=get_df_val(df,
                        'lab_sample',
                        {'cell_line_id_1000g': 'HG04217'}))

['/gpfs/projects/bsc83/Projects/pantranscriptome/novelannotations/espresso/39_IN5_HG04217_espresso.gtf']

## Old

In [65]:
meta_df = parse_config(meta_file)
print(len(meta_df.index))
meta_df.rename({'sample':'cell_line_id'}, axis=1, inplace=True)
meta_df.head()

43


,lab_rep,cell_line_id,tech_rep_num,tech_rep
0,10_NI5_GM19117,GM19117,1,GM19117_1
1,11_NI6_GM19129,GM19129,1,GM19129_1
2,12_NI7_GM19240,GM19240,1,GM19240_1
3,13_CH1_GM18542,GM18542,1,GM18542_1
4,14_CH2_GM18561,GM18561,1,GM18561_1


In [47]:
temp_df = pd.read_csv('cell_line_ids.txt', header=None, names=['cell_line_id'])

In [48]:
meta_df_2 = load_meta()
meta_df_2 = meta_df_2.loc[~meta_df_2['sample'].str.contains('_')]
meta_df_2[['cell_line_id', 'sample', 'hapmap_DNA_ID']].drop_duplicates()
meta_df = meta_df.merge(meta_df_2, 
                        how='left',
                        on='cell_line_id')
len(meta_df.index)

43

In [49]:
# meta_df = meta_df.loc[meta_df.cell_line_id.isin(temp_df['cell_line_id'].tolist())]
# len(meta_df.index)

In [50]:
len(meta_df['cell_line_id'].unique())

43

In [57]:
meta_df = meta_df.loc[(meta_df.cell_line_id.isin(temp_df.cell_line_id.tolist()))|
                (meta_df.hapmap_DNA_ID.isin(temp_df.cell_line_id.tolist()))]

30

In [58]:
# sample information
meta_file = '../config.tsv'
meta_df = parse_config(meta_file)
# meta_df.rename({'sample':'cell_line_id'}, axis=1, inplace=True)

# # info about the ref files
# temp_df = pd.read_csv('cell_line_ids.txt', header=None, names=['cell_line_id'])

# meta_df_2 = load_meta()
# meta_df_2 = meta_df_2.loc[~meta_df_2['sample'].str.contains('_')]
# meta_df_2[['cell_line_id', 'sample', 'hapmap_DNA_ID']].drop_duplicates()
# meta_df = meta_df.merge(meta_df_2,
#                         how='left',
#                         on='cell_line_id')

# # limit to 1000g samples
# meta_df = meta_df.loc[(meta_df.cell_line_id.isin(temp_df.cell_line_id.tolist()))|
#                 (meta_df.hapmap_DNA_ID.isin(temp_df.cell_line_id.tolist()))]

# hap = ['hap1', 'hap2']
# print(len(meta_df.index))

30


In [60]:
meta_df = meta_df.merge(temp_df,
                        how='left', 
                        on='cell_line_id')
meta_df = meta_df.merge(temp_df,
                        how='left',
                        left_on='hapmap_DNA_ID',
                        right_on='cell_line_id')

60

In [61]:
meta_df.head()

,lab_rep,cell_line_id_x,tech_rep_num,tech_rep,lab_number_sample,lab_sampleid,sample,population,color_pop,ooa,...,two_runs,was_switched_with,relabeled,family_member,family,population_full_name,population _description,extra_info,coriell_link,cell_line_id_y
0,10_NI5_GM19117,GM19117,1,GM19117_1,10,NI5,YRI5,YRI,#DFBD00,AFR,...,False,FALSE,False,Father,Y100,"Yoruba in Ibadan, Nigeria","Yoruba in Ibadan, Nigeria",Empty,https://www.coriell.org/0/Sections/Search/Samp...,NA19117
1,10_NI5_GM19117,GM19117,1,GM19117_1,10,NI5,YRI5,YRI,#DFBD00,AFR,...,False,FALSE,False,Father,Y100,"Yoruba in Ibadan, Nigeria","Yoruba in Ibadan, Nigeria",Empty,https://www.coriell.org/0/Sections/Search/Samp...,NA19117
2,11_NI6_GM19129,GM19129,1,GM19129_1,11,NI6,YRI6,YRI,#DFBD00,AFR,...,False,FALSE,False,Child,Y077,"Yoruba in Ibadan, Nigeria","Yoruba in Ibadan, Nigeria",Empty,https://www.coriell.org/0/Sections/Search/Samp...,NA19129
3,11_NI6_GM19129,GM19129,1,GM19129_1,11,NI6,YRI6,YRI,#DFBD00,AFR,...,False,FALSE,False,Child,Y077,"Yoruba in Ibadan, Nigeria","Yoruba in Ibadan, Nigeria",Empty,https://www.coriell.org/0/Sections/Search/Samp...,NA19129
4,12_NI7_GM19240,GM19240,1,GM19240_1,12,NI7,YRI7,YRI,#DFBD00,AFR,...,False,FALSE,False,Child,Y117,"Yoruba in Ibadan, Nigeria","Yoruba in Ibadan, Nigeria",Empty,https://www.coriell.org/0/Sections/Search/Samp...,NA19240


In [51]:
# print(len(meta_df.loc[meta_df['1000G']==True]['cell_line_id'].unique()))
# print(len(meta_df.loc[(meta_df['1000G']==True)&
#                       (meta_df['1000g_cell_line_id_x'].isnull())&\
#                       (meta_df['1000g_cell_line_id_y'].notnull())]['cell_line_id'].unique()))
# print(len(meta_df.loc[(meta_df['1000G']==True)&
#                       (meta_df['1000g_cell_line_id_y'].isnull())&\
#                       (meta_df['1000g_cell_line_id_x'].notnull())]['cell_line_id'].unique()))
# print(len(meta_df.loc[(meta_df['1000G']==True)&
#                       (meta_df['1000g_cell_line_id_y'].notnull())|\
#                       (meta_df['1000g_cell_line_id_x'].notnull())]['cell_line_id'].unique()))


In [55]:
def get_df_val(df, col1, col_dict):
    temp = df.copy(deep=True)

    for key, item in col_dict.items():
        if type(key) == tuple:
            temp2 = pd.DataFrame()
            for i in range(len(key)):
                sub_key = key[i]
                temp2 = pd.concat([temp2, temp.loc[temp[sub_key]==item]], axis=0)
            temp = temp2.drop_duplicates()
        else:
            temp = temp.loc[temp[key] == item]

    val = temp[col1].unique()
    assert len(val) == 1
    return val[0]

In [56]:
for thing in temp_df['cell_line_id'].unique():
    print(get_df_val(meta_df, 'sample', {('cell_line_id', 'hapmap_DNA_ID'):thing}))

PEL3
PEL5
YRI6
ITU3
ITU2
YRI5
CEU5
YRI3
LWK2
LWK1
YRI2
YRI1
HAC1
YRI7
LWK5
CEU1
ITU1
HAC3
LWK4
HAC6
PEL2
HAC2
CEU2
CEU3
PEL4
ITU5
PEL6
PEL1
ITU4
CEU4
